# Binary Search Trees

This is the code ot go along with the video explanation. Check out the video lecture for full details!

這裡的key , val有什麼差呢？

key是用來比對，抓出node要擺放的位置，

value就 真的是它的值。

        def findSuccessor(self):
       
        succ = None
        if self.hasRightChild():
            succ = self.rightChild.findMin()
        else:
            if self.parent:
               
                if self.isLeftChild():
                   
                    succ = self.parent
                else:
                    self.parent.rightChild = None
                    succ = self.parent.findSuccessor()
                    self.parent.rightChild = self
        return succ
 
這裡要講的是，沒有右子樹的情況，
 
沒有右子樹後，　第一個條件是看有沒有parent，這很重要，因為，
 
不管currentNode是屬於左子樹或右子樹，都需要往上去看parent，
 
以左子樹為例，回傳的successor就是它的parent，而右子樹的情況，是最麻煩的，
 
它還有用到，Recursive，第一行是把 parent.rightChild　給 None，
 
這是為了避免，弟回的時侯，會以為有右子樹，而跑去找他最小的Node，
 
右子樹的parent往回推的話，就要一直看，parent是在左子樹還是右子樹，
 
左子樹的情況，就回傳它的parent，右子樹的話，只能一直往回看，直到遇到ＲＯＯＴ，
 
self.parent.rightChild = self　最後這一行是做還原的動作，
 
因為一開始，self.parent.rightChild = None　要弟回前，都先把自己parent指向自己的
 
rightChild設成None，找到要的Successor後，就要把這個rightChild的指標還原。


In [5]:
class TreeNode:
    
    def __init__(self,key,val,left=None,right=None,parent=None):
        self.key = key
        self.payload = val
        self.leftChild = left
        self.rightChild = right
        self.parent = parent

    def hasLeftChild(self):
        return self.leftChild

    def hasRightChild(self):
        return self.rightChild

    # self.parent ，要有parent, 後面的判斷式，才有看的必要
    def isLeftChild(self):
        return self.parent and self.parent.leftChild == self

    def isRightChild(self):
        return self.parent and self.parent.rightChild == self

    # 依有沒有parent 來判斷 自已是不是root 
    def isRoot(self):
        return not self.parent

    # 這個 寫法不直觀 ，但可以這樣想，or 的回傳True的話 ，
    # 三種 情況  10 01 11 , 也就 是說二個子node只要有值，就 是回傳True
    # 所 以前前面才要一個not  
    def isLeaf(self):
        return not (self.rightChild or self.leftChild)

    def hasAnyChildren(self):
        return self.rightChild or self.leftChild

    def hasBothChildren(self):
        return self.rightChild and self.leftChild

    # lc rc 是leftchild , rightchild的意思嗎？
    def replaceNodeData(self,key,value,lc,rc):
        self.key = key
        self.payload = value
        self.leftChild = lc
        self.rightChild = rc
        # 這裡的if 判斷 ，我不懂？
        if self.hasLeftChild():
            self.leftChild.parent = self
        if self.hasRightChild():
            self.rightChild.parent = self


class BinarySearchTree:

    def __init__(self):
        self.root = None
        self.size = 0

    def length(self):
        return self.size

    # 這個內建func , 是不是用在 len(object)
    def __len__(self):
        return self.size

    # root有值，但不知道尾巴在那，這顆樹不是用list實作，所以不能用 append,
    # 所 以，我估計, _put是用來抓尾巴的，哈哈，猜錯，感覺就只是分成二步，
    # 一步用 root 去判斷 ，一步用 bst property去分。
    def put(self,key,val):
        if self.root:
            self._put(key,val,self.root)
        else:
            self.root = TreeNode(key,val)
        self.size = self.size + 1

    # val算是content ，而 key是用來決定， 你要被放在那裡。
    # 這很重要，用來維持，BinarySearchTree的結構，vedio裡有講到一點，
    # 就是有duplicate key的情況，但這裡沒有check這塊。
    def _put(self,key,val,currentNode):
        if key < currentNode.key:
            if currentNode.hasLeftChild():
                   self._put(key,val,currentNode.leftChild)
            else:
                   currentNode.leftChild = TreeNode(key,val,parent=currentNode)
        else:
            if currentNode.hasRightChild():
                   self._put(key,val,currentNode.rightChild)
            else:
                   currentNode.rightChild = TreeNode(key,val,parent=currentNode)

    # 這個func 要怎麼呼叫，怎麼使用
    # Reference:
    # http://www.diveintopython.net/object_oriented_framework/special_class_methods.html
    # 最簡單 aaa['a'] = b 這就 是setitem, 'a'是key , b 是值。
    def __setitem__(self,k,v):
        self.put(k,v)

    # 如果找不到 key的話 ，怎麼伴
    def get(self,key):
        if self.root:
            res = self._get(key,self.root)
            if res:
                
                return res.payload
            else:
                return None
        else:
            return None

    def _get(self,key,currentNode):
        
        if not currentNode:
            return None
        elif currentNode.key == key:
            return currentNode
        # 依key去分辦 ，往左或往右走，
        elif key < currentNode.key:
            return self._get(key,currentNode.leftChild)
        else:
            return self._get(key,currentNode.rightChild)

    # vedio裡有說，這個special method ，是可以用brackets 來抓值的。
    def __getitem__(self,key):
        return self.get(key)

    # Reference:
    # https://stackoverflow.com/questions/1964934/what-does-contains-do-what-can-call-contains-function
    # 藉由呼叫 _get func去樹裡面找，看key有沒有在裡面。
    def __contains__(self,key):
        if self._get(key,self.root):
            return True
        else:
            return False

    def delete(self,key):
        
        # 為什麼要大於1 ，等於1 也可以刪丫，對，但做法不一樣
        if self.size > 1:
            
            nodeToRemove = self._get(key,self.root)
            if nodeToRemove:
                # 大概有想法了，為什麼要多一個remove method,
                # 因為要被remove的node，有三種情況，Isleaf or 有一個child or 
                # 有二個child
                self.remove(nodeToRemove)
                self.size = self.size-1
            else:
                raise KeyError('Error, key not in tree')
        # 只有root 一個node的時侯 ，做的操作是不一樣的。
        elif self.size == 1 and self.root.key == key:
            self.root = None
            self.size = self.size - 1
        else:
            raise KeyError('Error, key not in tree')
    # del list1[0]  很少用，不知道有這個special method
    def __delitem__(self,key):
        
        self.delete(key)

    # 這個真不知道，要幹麻的，找到successor後 ，便是這個function要開始做事了，
    # successor可能會有右子樹，scuccessor在搬家之前，要把它的子孫給照顧好。
    def spliceOut(self):
        if self.isLeaf():
            if self.isLeftChild():
                
                self.parent.leftChild = None
            else:
                self.parent.rightChild = None
        elif self.hasAnyChildren():
            if self.hasLeftChild():
                
                if self.isLeftChild():
                    
                    self.parent.leftChild = self.leftChild
                else:
                    
                    self.parent.rightChild = self.leftChild
                    self.leftChild.parent = self.parent
        else:
                    
            if self.isLeftChild():
                        
                self.parent.leftChild = self.rightChild
            else:
                self.parent.rightChild = self.rightChild
                self.rightChild.parent = self.parent

    # 當一個node被刪，而它有左右子樹時，就會要用到這個function了。
    # 這種情況，都是要找右子樹中，最小的那個，來頂替被刪node的位置 
    # 而這個node就叫successor.
    # Reference
    # http://www.geeksforgeeks.org/inorder-successor-in-binary-search-tree/
    # 原來在Inorder的BST，要找successor也有分三種情況，第一種有右子樹，
    # 直接抓右子樹中，最小的那個node，接下來的二個case，是leaf的case，
    # 而這種 case分成，currentNode在左子樹or 在右子樹，currentNode在左子樹的話 ，
    # successor就是回傳它的parent, 第二個在右子樹的情況，只能一直往parent找，
    # 第一個比自已大的值，就 是我們要找的。
    def findSuccessor(self):
        
        succ = None
        if self.hasRightChild():
            succ = self.rightChild.findMin()
        # 這個class的使用，不會走到下面，而作者的寫法是完整的，
        # 它把currentNode是leaf，且是在左子樹or右子樹的情況，也一併寫進來了。
        else:
            if self.parent:
                
                if self.isLeftChild():
                    # 比左子樹大一點點的，就是它的parent了。
                    succ = self.parent
                else:
                    self.parent.rightChild = None
                    succ = self.parent.findSuccessor()
                    self.parent.rightChild = self
        return succ

    # 最小值一定在左子樹最小的leaf ，所 以往左直走下去，就 對了。
    # 往左走到底，找到的node，一定沒有左子樹，但可能會有右子樹。
    def findMin(self):
        
        current = self
        while current.hasLeftChild():
            current = current.leftChild
        return current

    # 奇怪了，這裡的remove 跟上面的delete有什麼差呢？
    # 這裡的remove , 要考慮三種 情況，那就 是下面有沒有 child,
    # 情況就 三種 ，10 01 00 11 , 10 01算一種 ，所 以總 共三種 
    def remove(self,currentNode):
        
        # 最簡單的一種，下面沒有子孫， 所 以只要依這個node，看是左還是右
        # node，把指向它的reference 設成None就 可以了。
        if currentNode.isLeaf(): #leaf
            if currentNode == currentNode.parent.leftChild:
                currentNode.parent.leftChild = None
            else:
                currentNode.parent.rightChild = None
        elif currentNode.hasBothChildren(): #interior
            
            succ = currentNode.findSuccessor()
            succ.spliceOut()
            currentNode.key = succ.key
            currentNode.payload = succ.payload

        else: # this node has one child
            # 好麻煩喔，先判斷 自已是有左還是右子樹，然後 還要判斷 ，
            # 自已是parent的左還是右子樹。
            if currentNode.hasLeftChild():
                if currentNode.isLeftChild():
                    # 都在同一邊的，讓 你的parent指向你的左子樹，
                    # 再讓 左子樹的parent指向自已的parent
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.leftChild
                elif currentNode.isRightChild():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.leftChild
                # 這個else 我不太了解怎麼回事哎，案，我懂了，自已沒有parent時，
                # 代表自已是root，而且又沒有右子樹，那就 把左子樹拉上來，當root,
                # 就 可以了。
                else:
                
                    currentNode.replaceNodeData(currentNode.leftChild.key,
                                    currentNode.leftChild.payload,
                                    currentNode.leftChild.leftChild,
                                    currentNode.leftChild.rightChild)
            else:
                
                if currentNode.isLeftChild():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.rightChild
                elif currentNode.isRightChild():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.rightChild
                else:
                    currentNode.replaceNodeData(currentNode.rightChild.key,
                                    currentNode.rightChild.payload,
                                    currentNode.rightChild.leftChild,
                                    currentNode.rightChild.rightChild)




In [6]:
mytree = BinarySearchTree()
mytree[3]="red"
mytree[4]="blue"
mytree[6]="yellow"
mytree[2]="at"

print(mytree[6])
print(mytree[2])

yellow
at


** Check the video for full explanation! **